In [1]:
import numpy as np
import cv2
import tensorflow as tf
import time

import configparser
config = configparser.ConfigParser()
config.read("config.ini")

# from test import *
# from models.utilities import *
from models.subclasses import *
from models.predict import *

KeyboardInterrupt: 

In [ ]:
# Static values
PATH = 'testVideos/IMG_9367.MOV'

test_image, _ = load_image("testVideos/35506150_cbdb630f4f.jpg")

test_image2, _ = load_image("testVideos/IMG_3004.jpg")

#how many frame to play until pause
SHOW_FRAME = 60

units = int(config['config']['units'])
embedding_dim = int(config['config']['embedding_dim'])

In [ ]:
img_name_train, cap_train, vocabulary, tokens_shape = load_dataset()
word_to_index, index_to_word = index_vocab(vocabulary)

In [ ]:
# temp = tf.convert_to_tensor(
#     word_to_index
# )
# temp('<start>')

In [ ]:
train_image_paths, image_path_to_caption_train = import_files(shuffle= False, method = "train")

train_captions = []
img_name_vector_train = []
for image_path in train_image_paths:
  caption_list = image_path_to_caption_train[image_path]
  train_captions.extend(caption_list)
  img_name_vector_train.extend([image_path] * len(caption_list))

60000


In [ ]:
word_to_index_train, index_to_word_train, tokenizer_train, cap_vector_train = tokenization(train_captions, max_length, 400000)

In [ ]:
embeddings_index = {}

if use_glove:
    glove_path = "./dataset/glove.6B/glove.6B.100d.txt"

    
    with open(glove_path, encoding="utf8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs

    print("Found %s word vectors." % len(embeddings_index))

    vocabulary = tokenizer_train.get_vocabulary()
    word_index = dict(zip(vocabulary, range(len(vocabulary))))

    num_tokens = len(vocabulary) + 2
    embedding_dim = 100
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 17160 words (15228 misses)


In [ ]:
encoder = CNN_Encoder(embedding_dim)
if use_glove:
    decoder = RNN_Decoder(embedding_dim, units, num_tokens, embedding_matrix)
else:
    decoder = RNN_Decoder(embedding_dim, units, tokenizer_train.vocabulary_size(), None)
image_features_extract_model = get_feature_extractor()

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
ckpt.restore(ckpt_manager.latest_checkpoint)

In [ ]:
image_features_extract_model = get_feature_extractor()

In [ ]:
def predict_all(frame):
    # img = tf.keras.layers.Resizing(224, 224)(frame)
    # img = tf.keras.applications.resnet50.preprocess_input(img)
    
    img = tf.keras.layers.Resizing(224, 240)(frame)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    result = predict_image(img, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)
    temp = Image.fromarray(frame, 'RGB')
#     temp.show()
    print(result)
    return result

In [43]:
testing, _ = load_image("testVideos/17-04-04_22-10-16_AD2F.jpg")

In [48]:
predict_image(testing, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)

['a',
 'suitcase',
 'filled',
 'with',
 'balloons',
 'attached',
 'to',
 'two',
 'snow',
 'wheels',
 'on',
 'the',
 'skiis',
 '<end>']

In [47]:
predict_image(test_image2, encoder, decoder,
            image_features_extract_model,
            word_to_index, index_to_word)

['a', 'green', 'pasture', 'with', 'road', 'with', 'trees.', '<end>']

## Run the app

In [1]:
#loop over all frames  'space' = next frame | 'q' = quit
cap = cv2.VideoCapture(PATH)
if (cap.isOpened()== False):
    print("Error opening video stream or file")
else:
    print("Statred capturing")

total_frames = 0
while cap.isOpened():
    HasFrames, frame = cap.read()
    #if vidoe is not done do
    if HasFrames:
        total_frames += 1
        cv2.imshow('Video', frame)
        #when you reach the pause frame do
        if((total_frames % SHOW_FRAME) == 0):
            #Caption 'frame'
            start = time.time()
            # print(frame.shape)
            predict_all(frame)
            print(f'Time taken for 1 image {time.time()-start:.4f} sec\n')
            #press 'E' to get next frame
            if(cv2.waitKey(5000) == ord('e')):
                continue

        if(cv2.waitKey(25) == ord('q')):
            break    

    else:
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined